In [1]:
!pip install transformers==4.41.2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [3]:
cihaz = "cuda"

In [7]:
model_adi = "kayrab/ytu_doktor_gpt2-medium"

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_adi)
model = AutoModelForCausalLM.from_pretrained(model_adi)

tokenizer_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [11]:
import ipywidgets as widgets
from IPython.display import display, HTML

In [12]:
generation_config = GenerationConfig(
    max_length=450,       # Üretilecek metnin maksimum uzunluğu
    num_beams=13,          # Beam search kullanarak üretilen beam sayısı
    no_repeat_ngram_size=3,  # Tekrarlanan n-gramların engellenmesi
    temperature=0.5,      # Rastgelelik seviyesini kontrol eden sıcaklık yüksek rastgele demek
    top_k=50,             # En olası k kelimenin seçileceği sampling yöntemi yüksek rastgele demek
    top_p=0.5,           # Nükleus (top-p) sampling oranı yüksek rastgele
    do_sample=True,       # Sampling yapılıp yapılmayacağı (False ise rastgelelik yok)
)

In [13]:
def generate_text(full_prompt_text, prompt_text, tokenizer, model, device, generation_config):
    # Modeli belirtilen cihaza gönder
    model.to(device)

    # Girdi metnini tokenize et ve cihazda kullanılabilir hale getir
    input_ids = tokenizer(full_prompt_text, return_tensors="pt").input_ids.to(device)

    # Model ile metin üretme
    output_ids = model.generate(input_ids, **generation_config.to_dict())
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Girdi metnini üretilen metinden çıkarma
    generated_response = generated_text[len(prompt_text) + 2:].strip()
    return generated_response
def apply_chat_template(message, doctor_title, doctor_speciality):
    user_message = f"<|USER|> {message}\n"
    doctor_info = f"<|DOCTOR_TITLE|> {doctor_title} <|SPECIALITY|> {doctor_speciality}\n"
    assistant_message = f"<|ASSISTANT|>"
    return f"{user_message}\n{doctor_info}\n{assistant_message}"

def test_et(tokenizer, model, device, titles, specialities, generation_config):
    def on_submit(change):
        speciality_dropdown.disabled = True
        title_dropdown.disabled = True
        text_box.disabled = True
        submit_button.disabled = True
        prompt_text = text_box.value
        if prompt_text == '0':
            return
        full_prompt_text = apply_chat_template(prompt_text, title_dropdown.value, speciality_dropdown.value)
        generated_text = generate_text(full_prompt_text, prompt_text, tokenizer, model, device, generation_config)
        display(HTML(f"\n\n<strong>Soru:</strong> {prompt_text}"))
        display(HTML(f"\n<strong>{generated_text}</strong>\n\n"))
        speciality_dropdown.disabled = False
        title_dropdown.disabled = False
        text_box.disabled = False
        submit_button.disabled = False

    # Doktor unvanları ve uzmanlık alanları
    selected_title = ""
    selected_speciality = ""

    # Widgets tanımlama
    speciality_dropdown = widgets.Dropdown(options=specialities, description='Uzmanlık alanı:', disabled=False)
    title_dropdown = widgets.Dropdown(options=titles, description='Doktor unvanı:', disabled=False)
    text_box = widgets.Text(description='Soru:', placeholder='Sorunuzu yazın 0 durdurur')
    submit_button = widgets.Button(description='Gönder')

    # Gönder butonuna tıklama olayını bağlama
    submit_button.on_click(on_submit)

    # Widgets'ları görüntüleme
    display(speciality_dropdown)
    display(title_dropdown)
    display(text_box)
    display(submit_button)

# Ünvan Alan Çiftleri

In [14]:
titles = [
"Dr. Dt.",
"Prof. Dr. Dt.",
"Uzm. Dyt.",
"Dr. Psk.",
"Uzm. Psk.",
"Dr.",
"Psk.",
"Dil ve Konuşma Terapisti",
"Uzm. Psk. Dan.",
"Dr. Öğr. Üyesi Dt.",
"Klinik Psikolog ",
"Dr. Dyt. ",
"Yrd. Doç. Dr.",
"Uzm. Dr. Dt.",
"Çocuk Gelişim Uzmanı",
"Doç. Dr. Psk. Dan",
"Aile Danışmanı",
"Pedagog",
"Uzm. Kl. Psk.",
"Uzm. Dr.",
"Psikoterapist",
"Uzm. Dt.",
"Op. Dr.",
"Doç. Dr. Dt.",
"Ergoterapist",
"Dr. Psk. Dan.",
"Prof. Dr.",
"Dt.",
"Fzt.",
"Dr. Öğr. Üyesi",
"Doç. Dr.",
"Dyt.",
"Vet.",
"Dr.Öğr.Üyesi",
"Psk. Dan.",
"Veteriner Hekim"
]
titles = sorted(titles)

specialities = [
"cocuk-sagligi-ve-hastaliklari-cocuk-genetik-hastaliklari",
"acil-tip",
"cocuk-gelisim-uzmani",
"pratisyen-hekimlik",
"cocuk-sagligi-ve-hastaliklari-cocuk-gogus-hastaliklari",
"aile-danismani",
"genel-cerrahi-endokrin-cerrahisi",
"histoloji-ve-embriyoloji",
"kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite",
"aile-hekimligi",
"fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-romatoloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-nefrolojisi",
"romatoloji-dahiliye-ve-ic-hastaliklari",
"uroloji",
"enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji",
"tibbi-biyokimya",
"dis-hekimi-koruyucu-dis-hekimlig",
"radyasyon-onkolojisi",
"cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi-cocuk-urolojisi-uroloji",
"genel-cerrahi-proktoloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-hematoloji",
"dis-hekimi-restoratif-dis-tedavisi",
"fizyoterapi",
"cocuk-cerrahisi",
"ortopedi-ve-travmatoloji",
"tibbi-patoloji",
"fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-algoloji",
"kadin-hastaliklari-ve-dogum-perinatoloji",
"dahiliye-ve-ic-hastaliklari-hematoloji",
"genel-cerrahi-cerrahi-onkoloji",
"dahiliye-ve-ic-hastaliklari-gastroenteroloji",
"dahiliye-ve-ic-hastaliklari-immunoloji",
"kalp-damar-cerrahisi-damar-cerrahisi",
"radyoloji-girisimsel-radyoloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-immunolojisi-ve-alerji-hastaliklari",
"ergoterapi",
"veteriner",
"cocuk-sagligi-ve-hastaliklari-cocuk-romatolojisi",
"dermatoloji",
"cocuk-sagligi-ve-hastaliklari",
"psikolojik-danisman-psikoloji",
"geleneksel-ve-tamamlayici-tip",
"cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi",
"dahiliye-ve-ic-hastaliklari-nefroloji",
"cocuk-sagligi-ve-hastaliklari-cocuk-enfeksiyon-hastaliklari",
"noroloji-klinik-norofizyoloji",
"klinik-psikolog-psikoloji",
"psikolojik-danisman",
"psikoterapi",
"dahiliye-ve-ic-hastaliklari",
"dis-hekimi-oral-implantoloji",
"kadin-hastaliklari-ve-dogum-jinekolojik-onkoloji",
"uroloji-androloji",
"genel-cerrahi-meme-cerrahisi",
"cocuk-sagligi-ve-hastaliklari-cocuk-kardiyolojisi",
"genel-cerrahi-gastroenteroloji-cerrahisi",
"dis-hekimi-ortodonti",
"algoloji",
"klinik-psikolog-psikoloji-aile-danismani-psikolog",
"dahiliye-ve-ic-hastaliklari-endokrinoloji-ve-metabolizma-hastaliklari",
"spor-hekimligi",
"cocuk-sagligi-ve-hastaliklari-neonatoloji",
"akupunktur-biorezonans",
"dis-hekimi-agiz-dis-ve-cene-cerrahisi",
"kardiyoloji",
"endokrinoloji-ve-metabolizma-hastaliklari",
"medikal-estetik-sertifikali-tip-doktoru",
"genel-cerrahi",
"dis-hekimi-periodontoloji",
"plastik-rekonstruktif-ve-estetik-cerrahi-el-cerrahisi-ve-mikrocerrahi-plastik",
"kadin-hastaliklari-ve-dogum",
"dil-konusma-bozuklari-b",
"dis-hekimi-pedodonti",
"tibbi-onkoloji-dahiliye-ve-ic-hastaliklari",
"cocuk-sagligi-ve-hastaliklari-cocuk-gastroenterolojisi",
"cocuk-sagligi-ve-hastaliklari-cocuk-endokrinoloji",
"klinik-psikolog",
"tibbi-onkoloji",
"diyetisyen",
"radyoloji-uroradyoloji",
"dis-hekimi-oral-diagnoz-ve-radyoloji",
"tibbi-farmakoloji",
"anestezi-ve-reanimasyon",
"pedodonti",
"akupunktur",
"tibbi-genetik",
"noroloji",
"dahiliye-ve-ic-hastaliklari-immunoloji-ve-alerji-hastaliklari",
"plastik-rekonstruktif-ve-estetik-cerrahi",
"radyoloji",
"dis-hekimi-endodonti",
"goz-hastaliklari",
"gogus-cerrahisi",
"anestezi-ve-reanimasyon-algoloji-anestezi-ve-reanimasyon",
"psikoloji-aile-danismani-psikolog",
"kulak-burun-bogaz-hastaliklari",
"dahiliye-ve-ic-hastaliklari-geriatri",
"ortopedi-ve-travmatoloji-el-cerrahisi-ve-mikrocerrahi",
"pedagoji",
"gogus-hastaliklari",
"psikiyatri",
"beyin-ve-sinir-cerrahisi",
"dis-hekimi",
"gastroenteroloji",
"cocuk-psikiyatrisi",
"kalp-damar-cerrahisi",
"cocuk-sagligi-ve-hastaliklari-cocuk-norolojisi",
"anatomi",
"dil-konusma-bozuklari-b-aile-danismani",
"fiziksel-tip-ve-rehabilitasyon",
"psikoloji",
"noroloji-noroloji-algoloji",
"gogus-hastaliklari-alerji-ve-gogus-hastaliklari",
"psikoloji-psikoonkoloji",
"dis-hekimi-dis-protezi-uzmani",
"nukleer-tip"
]

specialities = sorted(specialities)

# Test

In [15]:
test_et(tokenizer, model, cihaz, titles, specialities, generation_config)

Dropdown(description='Uzmanlık alanı:', options=('acil-tip', 'aile-danismani', 'aile-hekimligi', 'akupunktur',…

Dropdown(description='Doktor unvanı:', options=('Aile Danışmanı', 'Dil ve Konuşma Terapisti', 'Doç. Dr.', 'Doç…

Text(value='', description='Soru:', placeholder='Sorunuzu yazın 0 durdurur')

Button(description='Gönder', style=ButtonStyle())

# Örnekler

**Dt.
dis-hekimi
60 yaşında babamın dişlerini yaptırmak istiyoruz. yardımcı olabilir misiniz?
tabı ben patnostayım buyrun yardımcı olmak ısterım**

Dt.
dis-hekimi
bir aydır ağzımda geçmeyen ufak bir yara var alt çenemde protezdenmi acaba ne yapmalıyım kötü bir hastalık olur diye korkmaya başladım lütfen beni aydınlatırmısınız meliha özkan
protez travmasına baglı epulıs gelısebılır yada patolojık bi durum olabılır bulunudugunuz yerde fakulte varsa cene hastalıkları yada cerrahısıne gorunmenız onerılır

Dr. Dt.
dis-hekimi
Merhaba dun dis cekimi yapildiktan sonra bu sabah asiri kanama ile hekimime basvurdum.Pihtinin olustugunu yanliz kanamanin durmadigi gerekcesiyle dikis islemi uyguladi.Bugun herhangi bir kanamam veya agrim yok yalniz dikis bolgesinde mukusa benzer bir doku olustu.Tekrar gitmem gerekir mi?
Hayır.. rahat olun.normal şeyler yazmıssiniz.. yara yerinden mukus gibi dokunun cikmasi normaldir..


**Dr. Dt.
dis-hekimi
Merhaba.1,5 ay önce köprü dişlerim yapistirildi. O zaman ağrı vs yoktu.ancak son 10-15 gündür sanki bir şey içince köprünün altına gidiyor hissi var. Böyle düşünmeme sebep ise kendi disim ile köprü olan disin arasında diğer dişlere nazaran boşluk olması. Örneğin dış arası fırça kullanırken o bölgeye boydan boya girebiliyor fırça. Baslarda aralarda yapıştırıcı kaldığından hissetmedim belki de.ayrica köprü altındaki diğer dişlerimde kanal yapılanlarda da ağrı hissettim. Dişçi kontrol etti röntgen çekti herhangi bir sorun göremedi. Ayrıca baş ve kulak ağrısı var. 20lik dislerle de ilgili sıkıntı yok .daha önce dişciden kaynaklı bi sorundan dolayı güvenim azaldı dişçiye karşı. Dişçi emin bir şekilde koprunun altına gitmez dedi ama ben de bu sebeplerden dolayı emin olamadım.cevaplariniz için simdiden teşekkürler.
Öncelikle geçmiş olsun. Köprü yapımından sonra bu tarz hassasiyetler olabilir . Geçicidir. Kanallı dişlerinizde - kanalın yeni yapıldığını farz ederek söylüyorum- de ilk zamanlar, belki bir aya varan hassasiyet hissetmeniz normaldir. Röntgen çekildiğini ve sorun görülmediğini söylüyorsunuz. 20 liklerinizde de sorun görülmediğine göre belki baş kulak ağrısı için KBB ye görünmekte fayda olabilir diye düşünüyorum. Baş kulak ağrısı dişe bağlı olsa dişte ya da dişlerde hassasiyetten daha fazla ağrı şikayetiniz olurdu.**

Dr. Dt.
dis-hekimi
Merhaba doktor bey bnm çocuğumun üst dişlerinde 7ayligindan beri çürüme oluyor tamamen o dişinin beyaz yeri kalmaktadırlar curudu o kadar doktora götürdük ama bi tedavisini yapamadılar hepsi başka bişey diyo neden kaynaklanıyor olabilir
bazen lekeler çürük izlenimi verebiliyor.muayenede ortaya çıkar. uğrarsanız bilgilendirebiliriz...saygılarımızla...

**Dr. Dt.
dis-hekimi
Merhaba doktor bey bnm çocuğumun üst dişlerinde 7ayligindan beri çürüme oluyor tamamen o dişinin beyaz yeri kalmaktadırlar curudu o kadar doktora götürdük ama bi tedavisini yapamadılar hepsi başka bişey diyo neden kaynaklanıyor olabilir
ÖNCELİKLE GEÇMİŞ OLSUN BİR SÜRÜ SEBEBİ OLABİLİR ilaça bağlı renkleşme uzun süre biboron kullanımı çok fazla antibiyotik**